#Ejercicio 53:
Si para un usuario tenemos la cantidad de acciones que realizó para cada tipo de log y la cantidad de veces que fue bloqueado: ¿Cuál es la acción que más y menos correlaciona con ser bloqueado? ¿Qué acción correlaciona más con saber algo (babel>=0) de inglés? (⭐⭐⭐)

Empiezo importando la librería pandas, cargando el primer archivo con el que voy a trabajar que es el de logs y repasar un poco de que se trata, la información de las columnas, la memoria que consume

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_logs = pd.read_csv('/content/drive/MyDrive/Datasets/logs.csv', usecols = ['item_id', 'contributor_username', 'contributor_id', 'action', 'title'])

In [ ]:
df_logs.sample(10)

,item_id,contributor_username,contributor_id,action,title
5834437,7889148,Clarapedia,3322165.0,autocreate,Usuario:Clarapedia
561208,561209,Cratón,176790.0,move,Pastores prezioso
6443242,8786801,Maintenance script,3643376.0,renameuser,Usuario:Jdomenech77
7774948,10571417,Jjmcarrascosa,4415601.0,create,Usuario:Jjmcarrascosa
5621046,7632983,Antur,125243.0,delete,Clímaco Sarmiento Ávila
1037622,1037623,Zero.rock,529774.0,create,Usuario:Zero.rock
2299720,2598996,Platonides,38328.0,delete,Ashanti Gold Sporting Club
1559970,1565546,Gabynelu,766567.0,create,Usuario:Gabynelu
11366507,14742932,Iylaq,6039191.0,autocreate,Usuario:Iylaq
3304037,4276060,Laura Fiorucci,96461.0,delete,MAL DE AMORES


In [ ]:
df_logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12188946 entries, 0 to 12188945
Data columns (total 5 columns):
 #   Column                Dtype  
---  ------                -----  
 0   item_id               int64  
 1   contributor_username  object 
 2   contributor_id        float64
 3   action                object 
 4   title                 object 
dtypes: float64(1), int64(1), object(3)
memory usage: 465.0+ MB


Ahorro una gran cantidad de memoria pasando la columna al tipo category, hago esto ya que las acciones son limitadas

In [ ]:
df_logs.action = df_logs.action.astype("category")
df_logs.memory_usage()

Index                        128
item_id                 97511568
contributor_username    97511568
contributor_id          97511568
action                  12190386
title                   97511568
dtype: int64

Veo que hay usuarios sin nombre ni id entonces decido eliminarlos ya que considero invalida esta informacion

In [ ]:
df_logs.loc[df_logs["contributor_username"].isna() & df_logs["contributor_id"].isna()]

,item_id,contributor_username,contributor_id,action,title
42406,42407,NaN,NaN,block,NaN
42416,42417,NaN,NaN,unblock,NaN
57522,57523,NaN,NaN,delete,W/index.php
79235,79236,NaN,NaN,move,NaN
79236,79237,NaN,NaN,move,NaN
...,...,...,...,...,...
12188877,15587331,NaN,NaN,create,Global 24
12188881,15587335,NaN,NaN,create,Discusión:Defensa nacional espiritual
12188913,15587367,NaN,NaN,create,Chiquimula F.C.
12188915,15587369,NaN,NaN,create,Black Hawk Down (libro)


In [ ]:
df_logs.dropna(subset = ['contributor_username', 'contributor_id'], how = 'all', inplace = True)
df_logs.loc[df_logs["contributor_username"].isna() & df_logs["contributor_id"].isna()]

,item_id,contributor_username,contributor_id,action,title


Armo un nuevo dataframe en donde tengo a los usuarios que bloquearon a otro, asi puedo conseguir el usuario de los bloqueados.
Como los usuarios bloqueados estan en la columna `title` necesito conseguir sus nombres limpiando un poco esta columna, y descartando a las ips, ya que no son usuarios

In [ ]:
df_de_bloqueos = df_logs.loc[df_logs['action'] == "block"]
df_de_bloqueos = df_de_bloqueos.drop(labels = ["contributor_username", "contributor_id", "action"], axis = 1)
df_de_bloqueos.dropna(subset = "title", inplace = True)

In [ ]:
df_de_bloqueos["title"] = df_de_bloqueos.title.map(lambda x: x[8:])
df_de_bloqueos

,item_id,title
5,6,212.55.30.157
287,288,201.241.228.15
288,289,201.255.50.32
290,291,84.122.0.158
313,314,81.192.12.93
...,...,...
12188865,15587319,186.141.200.121
12188886,15587340,Galaxi066
12188909,15587363,Xx.tkpro.xx
12188924,15587378,181.167.104.152


In [ ]:
def es_valido_el_usuario(user):
  es_valido = False
  for caracter in user:
      if caracter.isalpha():
        es_valido = True
      if caracter == ":":
        return False

  return es_valido

df_de_bloqueos= df_de_bloqueos[df_de_bloqueos['title'].map(lambda x: es_valido_el_usuario(x))]
df_de_bloqueos = df_de_bloqueos.rename(columns = {"title": "contributor_username"})
df_de_bloqueos

,item_id,contributor_username
4015,4016,Sirvodka
4444,4445,Willy on Wheels~eswiki
4445,4446,Willy on Wheels~eswiki
4450,4451,Willy on Wheels~eswiki
5347,5348,Dodo tis
...,...,...
12188717,15587171,David Sacristán
12188747,15587201,IvanDePineda8287
12188756,15587210,JohnWayne454545
12188886,15587340,Galaxi066


In [ ]:
df_de_bloqueos["action"] = "was blocked"
df_de_bloqueos

,item_id,contributor_username,action
4015,4016,Sirvodka,was blocked
4444,4445,Willy on Wheels~eswiki,was blocked
4445,4446,Willy on Wheels~eswiki,was blocked
4450,4451,Willy on Wheels~eswiki,was blocked
5347,5348,Dodo tis,was blocked
...,...,...,...
12188717,15587171,David Sacristán,was blocked
12188747,15587201,IvanDePineda8287,was blocked
12188756,15587210,JohnWayne454545,was blocked
12188886,15587340,Galaxi066,was blocked


Concateno los dos dfs que tenia, el de logs y el de bloqueos.
En `action` si un usuario fue bloqueado aparece `was blocked`, asi al final se podra contar la cantidad de veces que un usuario fue bloqueado como se cuentan la cantidad de acciones de cada usuario.

In [ ]:
df_con_bloqueos = pd.concat([df_logs, df_de_bloqueos], ignore_index=True)
df_con_bloqueos = df_con_bloqueos.drop(labels = "title", axis = 1)
df_con_bloqueos

,item_id,contributor_username,contributor_id,action
0,1,Sabbut,2075.0,delete
1,2,Sabbut,2075.0,delete
2,3,Sabbut,2075.0,delete
3,4,Sanbec,7856.0,delete
4,5,Sanbec,7856.0,delete
...,...,...,...,...
12165363,15587171,David Sacristán,NaN,was blocked
12165364,15587201,IvanDePineda8287,NaN,was blocked
12165365,15587210,JohnWayne454545,NaN,was blocked
12165366,15587340,Galaxi066,NaN,was blocked


Cargo el segundo archivo con el que voy a trabajar que es el de languages y repasar un poco de que se trata, la información de las columnas, la memoria que consume

In [ ]:
df_languages = pd.read_csv('/content/drive/MyDrive/Datasets/languages.csv')

In [ ]:
df_languages.sample(10)

,babel_user,babel_lang,babel_level
4304,884278,en,1
33399,193474,pt,2
34198,5973834,pt,N
29663,1728414,it,1
32589,805,pt,1
32759,220520,pt,1
7642,39641,en,3
3587,403025,de,N
16475,97780,es,N
14815,1874230,es,3


In [ ]:
df_languages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35666 entries, 0 to 35665
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   babel_user   35666 non-null  int64 
 1   babel_lang   35662 non-null  object
 2   babel_level  35666 non-null  object
dtypes: int64(1), object(2)
memory usage: 836.0+ KB


In [ ]:
df_languages.memory_usage()

Index             128
babel_user     285328
babel_lang     285328
babel_level    285328
dtype: int64

In [ ]:
df_languages['babel_level'].unique()

array(['2', '0', '1', '3', '4', 'N', '5'], dtype=object)

In [ ]:
df_languages['babel_lang'].unique()

array(['FR', 'aa', 'ab', 'ace', 'acf', 'af', 'agr', 'akk', 'aln', 'alt',
       'am', 'an', 'ang', 'ar', 'arc', 'arn', 'ary', 'ase', 'ast', 'av',
       'avk', 'ay', 'az', 'azb', 'ba', 'ban', 'bar', 'bcl', 'be',
       'be-tarask', 'bew', 'bg', 'bi', 'bjn', 'bm', 'bn', 'bo', 'br',
       'bs', 'bug', 'ca', 'cak', 'cas', 'cbk', 'ceb', 'ch', 'cho', 'chr',
       'ckb', 'cmn', 'cnt', 'co', 'cr', 'cs', 'csb', 'csc', 'csn', 'cu',
       'cv', 'cy', 'da', 'de', 'de-AT', 'de-CH', 'de-at', 'din', 'diq',
       'dsb', 'dz', 'ee', 'egy', 'el', 'eml', 'en', 'en-CA', 'en-GB',
       'en-gb', 'eo', 'es', 'et', 'eu', 'ext', 'fa', 'fax', 'fi', 'fil',
       'fj', 'fo', 'fr', 'frc', 'frp', 'frr', 'fur', 'fy', 'ga', 'gag',
       'gan', 'gcr', 'gd', 'gl', 'gn', 'gom-Latn', 'gor', 'got', 'grc',
       'gsw', 'guc', 'gv', 'gym', 'hak', 'haw', 'hch', 'he', 'hi', 'hr',
       'hsb', 'ht', 'hu', 'hy', 'ia', 'id', 'ie', 'ig', 'ii', 'ike-Cans',
       'ike-Latn', 'ilo', 'io', 'is', 'ise', 'it', 'iu', 'ja', 'j

In [ ]:
df_languages['babel_lang'] = df_languages.babel_lang.astype('category')
df_languages['babel_lang'] = df_languages.babel_lang.map(lambda x: x.lower())

In [ ]:
df_languages['babel_lang'].unique()

array(['fr', 'aa', 'ab', 'ace', 'acf', 'af', 'agr', 'akk', 'aln', 'alt',
       'am', 'an', 'ang', 'ar', 'arc', 'arn', 'ary', 'ase', 'ast', 'av',
       'avk', 'ay', 'az', 'azb', 'ba', 'ban', 'bar', 'bcl', 'be',
       'be-tarask', 'bew', 'bg', 'bi', 'bjn', 'bm', 'bn', 'bo', 'br',
       'bs', 'bug', 'ca', 'cak', 'cas', 'cbk', 'ceb', 'ch', 'cho', 'chr',
       'ckb', 'cmn', 'cnt', 'co', 'cr', 'cs', 'csb', 'csc', 'csn', 'cu',
       'cv', 'cy', 'da', 'de', 'de-at', 'de-ch', 'din', 'diq', 'dsb',
       'dz', 'ee', 'egy', 'el', 'eml', 'en', 'en-ca', 'en-gb', 'eo', 'es',
       'et', 'eu', 'ext', 'fa', 'fax', 'fi', 'fil', 'fj', 'fo', 'frc',
       'frp', 'frr', 'fur', 'fy', 'ga', 'gag', 'gan', 'gcr', 'gd', 'gl',
       'gn', 'gom-latn', 'gor', 'got', 'grc', 'gsw', 'guc', 'gv', 'gym',
       'hak', 'haw', 'hch', 'he', 'hi', 'hr', 'hsb', 'ht', 'hu', 'hy',
       'ia', 'id', 'ie', 'ig', 'ii', 'ike-cans', 'ike-latn', 'ilo', 'io',
       'is', 'ise', 'it', 'iu', 'ja', 'jam', 'jbo', 'jv', 'ka', 

In [ ]:
df_languages.dropna(subset = 'babel_user', inplace = True)

In [ ]:
df_languages['knows language'] = df_languages['babel_level'].map(lambda x: x in ['2', '0', '1', '3', '4', 'N', '5'])

Creo un nuevo dataframe en donde solo tengo usuarios que sepan algo de ingles. Como ya se realizo el chequeo de si el usuario sabe hablar algo del idioma, me quedo con todos los usuarios que tengan en `babel_lang` ingles, considero las 3 variaciones de ingles que hay: `en`, `en-gb`, `en-ca`.

In [ ]:
df_ingles = df_languages.loc[(df_languages['babel_lang'] == 'en') | (df_languages['babel_lang'] == 'en-gb') | (df_languages['babel_lang'] == 'en-ca')]
df_ingles['babel_lang'].unique()

array(['en', 'en-ca', 'en-gb'], dtype=object)

In [ ]:
df_ingles = df_ingles.drop(labels = ['babel_lang','babel_level'], axis = 1)

Le cambio el nombre a la columna de `babel_user` asi puedo hacer un merge con `df_con_bloqueos`, donde tengo los user id con ese nombre de columna.

In [ ]:
df_ingles = df_ingles.rename(columns = {'babel_user': 'contributor_id', 'knows language': 'knows english'})
df_ingles

,contributor_id,knows english
3893,61378,True
3894,136933,True
3895,182072,True
3896,647542,True
3897,845088,True
...,...,...
13365,5513245,True
13366,925848,True
13367,301330,True
13368,4884255,True


In [ ]:
df_total = pd.merge(df_con_bloqueos, df_ingles, how = 'outer')
df_total

,item_id,contributor_username,contributor_id,action,knows english
0,1.0,Sabbut,2075.0,delete,True
1,2.0,Sabbut,2075.0,delete,True
2,3.0,Sabbut,2075.0,delete,True
3,95.0,Sabbut,2075.0,delete,True
4,107.0,Sabbut,2075.0,delete,True
...,...,...,...,...,...
12165826,NaN,NaN,44327.0,NaN,True
12165827,NaN,NaN,578063.0,NaN,True
12165828,NaN,NaN,1005473.0,NaN,True
12165829,NaN,NaN,1676216.0,NaN,True


Como despues voy a tener que agrupar por usuario y tengo datos de nombre de usuario y datos de id de usuario decido que los usuarios con id pero sin nombre van a tener un nombre llamado `username_id: "su id"`.
Esto lo hago para que al agrupar pueda hacerlo por la columna de `contributor_username`. Y a los usuarios de los que que no tenga informacion sobre si saben algo de ingles los asumo como que no saben.

In [ ]:
df_total['contributor_username'] = df_total['contributor_username'].fillna('username_id: ' + df_total['contributor_id'].astype(str))

In [ ]:
df_total['knows english'] = df_total['knows english'].fillna(False)

In [ ]:
df_total

,item_id,contributor_username,contributor_id,action,knows english
0,1.0,Sabbut,2075.0,delete,True
1,2.0,Sabbut,2075.0,delete,True
2,3.0,Sabbut,2075.0,delete,True
3,95.0,Sabbut,2075.0,delete,True
4,107.0,Sabbut,2075.0,delete,True
...,...,...,...,...,...
12165826,NaN,username_id: 44327.0,44327.0,NaN,True
12165827,NaN,username_id: 578063.0,578063.0,NaN,True
12165828,NaN,username_id: 1005473.0,1005473.0,NaN,True
12165829,NaN,username_id: 1676216.0,1676216.0,NaN,True


In [ ]:
df_total.loc[df_total['contributor_username'].isna()]

,item_id,contributor_username,contributor_id,action,knows english


Ahora agrupo por `contributor_username` por el motivo que explique arriba, tambien por `knows english` y por `action`. Para luego contar la cantidad de acciones por cada uno.

In [ ]:
df = df_total.groupby(["contributor_username","knows english", "action"]).agg({"action":"count"}).unstack(fill_value = 0)
df

action          \
action                                                          add addlink   
contributor_username                           knows english                  
! ! ! ! ! ! ! ! ! ! ! ! ! ! HAGGEⓇ? 123        False              0       0   
! ! ! Histoire hebdomadaire de la patrie ! ! ! False              0       0   
! Bikkit !                                     False              0       0   
! DanSkammelsrod !                             False              0       0   
! Gabriel !                                    False              0       0   
...                                                             ...     ...   
𝚕𝚒𝚝𝚢 𝚌𝚟18                                      False              0       0   
😂                                              False              0       0   
🤖                                              False              0       0   
𠔻𪚥                                             False              0       0   
𥐣                                              False              0       0   

                                                                               \
action                                                       autocreate block   
contributor_username                           knows english                    
! ! ! ! ! ! ! ! ! ! ! ! ! ! HAGGEⓇ? 123        False                  1     0   
! ! ! Histoire hebdomadaire de la patrie ! ! ! False                  1     0   
! Bikkit !                                     False                  1     0   
! DanSkammelsrod !                             False                  0     0   
! Gabriel !                                    False                  0     0   
...                                                                 ...   ...   
𝚕𝚒𝚝𝚢 𝚌𝚟18                                      False                  0     0   
😂                                              False                  1     0   
🤖                                              False                  1     0   
𠔻𪚥                                             False                  1     0   
𥐣                                              False                  1     0   

                                                                             \
action                                                       byemail change   
contributor_username                           knows english                  
! ! ! ! ! ! ! ! ! ! ! ! ! ! HAGGEⓇ? 123        False               0      0   
! ! ! Histoire hebdomadaire de la patrie ! ! ! False               0      0   
! Bikkit !                                     False               0      0   
! DanSkammelsrod !                             False               0      0   
! Gabriel !                                    False               0      0   
...                                                              ...    ...   
𝚕𝚒𝚝𝚢 𝚌𝚟18                                      False               0      0   
😂                                              False               0      0   
🤖                                              False               0      0   
𠔻𪚥                                             False               0      0   
𥐣                                              False               0      0   

                                                                             \
action                                                       create create2   
contributor_username                           knows english                  
! ! ! ! ! ! ! ! ! ! ! ! ! ! HAGGEⓇ? 123        False              0       0   
! ! ! Histoire hebdomadaire de la patrie ! ! ! False              0       0   
! Bikkit !                                     False              0       0   
! DanSkammelsrod !                             False              1       0   
! Gabriel !                                    False              0       0   
...                                                             ...     ...   
𝚕

In [ ]:
df.columns = df.columns.droplevel()

In [ ]:
df.columns.name = None

In [ ]:
df = df.reset_index()
df = df.set_index('contributor_username')

In [ ]:
df

,knows english,add,addlink,autocreate,block,byemail,change,create,create2,delete,...,thank,unblock,undelete,unprotect,unreview,update,upload,usergroups,was blocked,whitelist
contributor_username,,,,,,,,,,,,,,,,,,,,,
! ! ! ! ! ! ! ! ! ! ! ! ! ! HAGGEⓇ? 123,False,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
! ! ! Histoire hebdomadaire de la patrie ! ! !,False,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
! Bikkit !,False,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
! DanSkammelsrod !,False,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
! Gabriel !,False,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
𝚕𝚒𝚝𝚢 𝚌𝚟18,False,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
😂,False,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
🤖,False,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Una vez modificado el dataset para poder sacar la matriz de correlacion lo hago y asi veo cual es la accion que mas y menos correlacion tiene con ser bloqueado y con saber ingles.

In [ ]:
matriz_corr = df.corr()
matriz_corr

,knows english,add,addlink,autocreate,block,byemail,change,create,create2,delete,...,thank,unblock,undelete,unprotect,unreview,update,upload,usergroups,was blocked,whitelist
knows english,1.000000,4.649321e-03,-1.954838e-05,-0.005006,0.018552,0.007105,2.377594e-03,1.419790e-02,0.009457,2.482411e-02,...,0.061365,0.020495,4.657960e-03,1.412318e-02,9.070271e-03,6.010284e-03,0.040588,9.905474e-03,-0.004557,-1.546267e-05
add,0.004649,1.000000e+00,-3.824142e-07,-0.000345,0.003212,0.009102,-3.637282e-07,6.563697e-04,0.006684,9.741745e-03,...,0.007714,0.004521,8.888073e-01,3.940036e-03,1.038391e-01,2.789677e-01,0.001256,-3.826199e-07,0.005632,-3.024876e-07
addlink,-0.000020,-3.824142e-07,1.000000e+00,-0.000271,-0.000001,-0.000013,-2.455548e-07,3.119629e-05,-0.000028,-1.494756e-06,...,0.000083,-0.000001,-2.739778e-07,-1.121539e-06,-9.917379e-07,-1.245619e-06,-0.000005,-2.583087e-07,-0.000060,-2.042110e-07
autocreate,-0.005006,-3.445787e-04,-2.707130e-04,1.000000,-0.001154,-0.013281,6.852296e-04,-5.025912e-03,-0.027780,-1.376543e-03,...,-0.001611,-0.001490,-2.872890e-04,-1.151601e-03,8.886115e-05,-8.998478e-04,-0.005106,-2.708587e-04,-0.044722,-2.141326e-04
block,0.018552,3.212340e-03,-1.288195e-06,-0.001154,1.000000,0.080049,1.280258e-02,4.889577e-02,0.091960,6.948691e-01,...,0.167009,0.746418,-1.367072e-06,3.206969e-01,4.658773e-04,1.346832e-02,0.032337,2.234167e-01,0.010823,3.442041e-01
byemail,0.007105,9.101710e-03,-1.313013e-05,-0.013281,0.080049,1.000000,2.290328e-03,2.371849e-03,0.083877,7.488928e-02,...,0.020228,0.061415,-1.393409e-05,4.368264e-02,-5.043829e-05,1.425956e-02,0.004779,9.525155e-03,0.005332,1.608575e-01
change,0.002378,-3.637282e-07,-2.455548e-07,0.000685,0.012803,0.002290,1.000000e+00,1.386797e-03,0.026148,8.861361e-03,...,0.022257,0.016205,-2.605904e-07,6.080143e-03,-9.432785e-07,-1.184754e-06,0.000342,2.172856e-01,-0.000057,-1.942326e-07
create,0.014198,6.563697e-04,3.119629e-05,-0.005026,0.048896,0.002372,1.386797e-03,1.000000e+00,0.005196,7.016367e-02,...,0.149307,0.016235,5.761569e-04,1.197446e-02,6.648490e-03,2.995590e-03,0.002958,6.715200e-04,0.011859,2.319099e-03
create2,0.009457,6.683835e-03,-2.826644e-05,-0.027780,0.091960,0.083877,2.614810e-02,5.195737e-03,1.000000,9.278473e-02,...,0.023994,0.121670,-2.999722e-05,7.487710e-02,2.666424e-03,9.665047e-03,0.006434,3.791768e-02,0.010950,1.711988e-02
delete,0.024824,9.741745e-03,-1.494756e-06,-0.001377,0.694869,0.074889,8.861361e-03,7.016367e-02,0.092785,1.000000e+00,...,0.190207,0.598747,-2.202301e-07,3.763685e-01,9.127496e-05,1.566329e-02,0.056146,9.176995e-02,0.011873,2.420944e-02


In [ ]:
min_corr_was_blocked = matriz_corr['was blocked'].drop(['knows english', 'was blocked']).idxmin()
max_corr_was_blocked = matriz_corr['was blocked'].drop(['knows english', 'was blocked']).idxmax()

In [ ]:
min_corr_was_blocked

'autocreate'

In [ ]:
max_corr_was_blocked

'move_redir'

In [ ]:
max_corr_knows_english = matriz_corr['knows english'].drop(['knows english', 'was blocked']).idxmax()
max_corr_knows_english

'thank'